In [7]:
import os
import anthropic
import openai
from openai import OpenAI
import base64
import json
import requests
import pandas as pd
import random
from http import HTTPStatus
from dashscope import Generation
import dashscope
from dashscope import MultiModalConversation
import google.generativeai as genai


ImportError: cannot import name 'load_dotenv' from 'dotenv' (/home/edward/miniconda3/envs/vlm_assess/lib/python3.12/site-packages/dotenv/__init__.py)

In [2]:
claude_key = os.getenv("CLAUDE_KEY")
openai_key = ""
gemini_key = "" #os.getenv("GEMINI_KEY")
qwen_key = os.getenv("QWEN_KEY")
grok_key = "xai-GJfN2vwqNOAZ1yyUhIE0USS1xYvGrmVrDb8gM0ZQmUrya8VoXwteXLgyZ1gB79V85v1fWR6LN2CmP1uL"
print(claude_key)

## API VLM Query Helpers

In [4]:
def claude_review(filepath, prompt):

    with open(filepath, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode('utf-8')  # Ensure the image data is in base64 string format
    
    
    #check the media type of the image
    image_media_type = "image/jpeg" if filepath.endswith(".jpg") else "image/png"

    client = anthropic.Anthropic(
        api_key=""
    )
    response = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": image_media_type,
                            "data": image_data,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    )
    return response.content[0].text


In [19]:
import openai

def openai_review(filepath, prompt):
  
  openai.api_key = ""
  # Function to encode the image
  def encode_image(image_path):
    with open(image_path, "rb") as image_file:
      return base64.b64encode(image_file.read()).decode('utf-8')

  # Path to your image
  image_path = filepath

  # Getting the base64 string
  base64_image = encode_image(image_path)

  response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt,
          },
          {
            "type": "image_url",
            "image_url": {
              "url":  f"data:image/jpeg;base64,{base64_image}"
            },
          },
        ],
      }
    ],
  )

  return(response.choices[0].message.content)


In [11]:
def qwen_review():
    """Simple single round multimodal conversation call.
    """
    messages = [
        {
            "role": "user",
            "content": [
                {"image": "https://raw.githubusercontent.com/luyingz06/VLMsForInjuryAssessment/refs/heads/main/images/injured_man.jpg"},
                {"text": "What is happening in this image?"}
            ]
        }
    ]
    responses = MultiModalConversation.call(model='qwen-vl-max',
                                           messages=messages,
                                           stream=False)

    print(responses['output']['choices'][0]['message']['content'][0]['text'])


In [33]:
def gemini_review(filepath, prompt):
    #TODO: Implement Gemini review
    #https://ai.google.dev/gemini-api?gad_source=1&gclid=Cj0KCQjwj4K5BhDYARIsAD1Ly2r_KaDE-fC0T3O2E0sA0kqv2WdncwLTkye_XaRwJ2tWEHm3tw9o5rUaAv83EALw_wcB
    genai.configure(api_key=gemini_key)
    # initialize model
    model = genai.GenerativeModel("gemini-1.5-pro")
    # count number of tokens for image
    image_file = genai.upload_file(filepath)
    
    result = model.generate_content(
        [image_file, "\n\n", prompt]
    )
    return result.text


In [ ]:
def grok_review(filepath, prompt):
    pass

## Dataset Assessment

In [13]:
def gen_imagepath_list(dir_path):
    imagepath_list = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                imagepath_list.append(os.path.join(root, file))
    return imagepath_list


In [6]:
from PIL import Image
#downsample the image of this path to be 5mb or less with same aspect ratio
#width is always 1024
def resize_image(image_path):
    image = Image.open(image_path)
    width = 1024
    #height should be scaled to preserve aspect ratio
    height = int(width * image.height / image.width)
    image = image.resize((width, height))
    #downsample the image to be 5mb or less
    image.save(image_path, quality=95)


In [20]:
from create_dataset import LABEL_CLASSES, label_class_human_value_list, dict_sentences


def gen_dataset_elt(path, model):
    image_path = path
    id = path.split("/")[-1].split(".")[0]
    #image should be the url starting at the second folder in 
    image = "/".join(path.split("/")[2:])
    conversations = [
        {
            "from": "human",
            "value": "LEAVE BLANK"
        }
    ]

    trauma_labels = {
            "from": "gpt",
            "scene": [],
            "trauma_head": [],
            "trauma_torso": [],
            "trauma_lower_ext": [],
            "trauma_upper_ext": [],
            "alertness_ocular": [],
            "severe_hemorrhage": [],
            "value-dict": ""
        }
    value_dict = {
        "trauma_head": "",
        "trauma_torso": "",
        "trauma_lower_ext": "",
        "trauma_upper_ext": "",
        "alertness_ocular": "",
        "severe_hemorrhage": ""
    }

    for key in label_class_human_value_list.keys():
        full_prompt = " ".join(label_class_human_value_list[key])
        if model == "claude":
            print(label_class_human_value_list[key])
            trauma_labels[key] = claude_review(image_path, full_prompt)
            try:
                value_dict[key] = claude_review(image_path, dict_sentences[key][0])
            except:
                pass
        elif model == "openai":
            print(label_class_human_value_list[key])
            trauma_labels[key] = openai_review(image_path, full_prompt)
            try:
                value_dict[key] = openai_review(image_path, dict_sentences[key][0])
            except:
                pass
        elif model == "gemini":
            print(label_class_human_value_list[key])
            trauma_labels[key] = gemini_review(image_path, full_prompt)
            try:
                value_dict[key] = gemini_review(image_path, dict_sentences[key][0])
            except:
                pass
        elif model == "qwen":
            trauma_labels[key] = qwen_review(image_path, label_class_human_value_list[key])
        else:
            raise ValueError("Invalid model name")
        print(key, " scored")
    
    trauma_labels["value-dict"] = json.dumps(value_dict)
    conversations.append(trauma_labels)
    return {
        "id": id,
        "image": image,
        "conversations": conversations
    }
    

In [25]:
from time import sleep
from progress.bar import Bar

#loop through the images directory and have each of the vlms assess the images given a prompt
#save it to a csv
#it should have the image name, the prompt, the response from each vlms

imagepaths = gen_imagepath_list("/home/edward/VLMsForInjuryAssessment/all_training_data_no_humans")

dataset = []

#add a progress bar
with Bar("Processing images", max=len(imagepaths)) as bar:
    for path in imagepaths:
        #print a progress bar as we go through the iamges
        df = gen_dataset_elt(path, "openai")
        dataset.append(df)
        with open("dataset_gpt4o.json", "w") as f:
            json.dump(dataset, f, indent=4)
        sleep(.02)
        bar.next()
        print("finished ", path)

#save as json
with open("dataset_gpt4o.json", "w") as f:
    json.dump(dataset, f, indent=4)




["Describe the scene. Is there a person in the image and what's their condition?"]
scene  scored
['Describe the scene. Does the person have any head injuries? If yes, specify the type of injury.', 'Give a detailed description of the person. Are there any head injuries?', 'Does the person show any signs of head injuries?', 'There is a person in the center of the scene. Describe the person. Are there any head injuries?', 'Does the person in the scene have any head injuries?']
trauma_head  scored
['Describe the scene. Does the person have any torso injuries? If yes, specify the type of injury.', 'Give a description of the person. Are there any torso injuries?', 'Are there any signs of torso injuries?', 'There is a person in the center of the scene. Describe the person. Are there any injuries to their torso?', 'Does the person in the scene have any torso injuries?']
trauma_torso  scored
['Describe the scene. Does the person have any lower extremity injuries? If yes, specify the type of inj

#TODO:
1. Implement the API Calls
2. Do a cost assessment
3. Finetune the different levels of LLaVA 
4. LoRA the different levels of LLaVA
5. Run the mannequin images through everything
6. Analysis